In [1]:
from copy import deepcopy
from dataclasses import dataclass
from itertools import product as iter_product
from pprint import pprint


import pieces as pieces_mod
from assorted import ARBITRARILY_LARGE_VALUE, dev_print
from vector import Vector

In [2]:

STARTING_POSITIONS: tuple[tuple[pieces_mod.Piece]] = (
    (
        pieces_mod.Rook(color="B"),
        pieces_mod.Knight(color="B"),
        pieces_mod.Bishop(color="B"),
        pieces_mod.Queen(color="B"),
        pieces_mod.King(color="B"),
        pieces_mod.Bishop(color="B"),
        pieces_mod.Knight(color="B"),
        pieces_mod.Rook(color="B")
    ),
    (pieces_mod.Pawn(color="B"),)*8,
    (None,)*8,
    (None,)*8,
    (None,)*8,
    (None,)*8,
    (pieces_mod.Pawn(color="W"),)*8,
    (
        pieces_mod.Rook(color="W"),
        pieces_mod.Knight(color="W"),
        pieces_mod.Bishop(color="W"),
        pieces_mod.Queen(color="W"),
        pieces_mod.King(color="W"),
        pieces_mod.Bishop(color="W"),
        pieces_mod.Knight(color="W"),
        pieces_mod.Rook(color="W")
    )
)

@dataclass(frozen=True)
class Board_State():
    pieces_matrix: tuple[tuple[pieces_mod.Piece]]
    next_to_go: int = "W"
    # pieces_matrix: tuple[tuple[pieces_mod.Piece]] = STARTING_POSITIONS

    def print_board(self, print_function=pprint):
        print_function(
            list(map(
                lambda row: list(map(
                    lambda piece: None if piece is None else piece.symbol(),
                    row
                )),
                self.pieces_matrix
            ))
        )

    def get_piece_at_vector(self, vector: Vector):
        column, row = vector.i, 7-vector.j
        # column, row = vector.i, vector.j
        return self.pieces_matrix[row][column]

    def generate_all_pieces(self):
        # for i, j in zip(range(8), range(8)):
        # should be product
        for i, j in iter_product(range(8), range(8)):
            position_vector = Vector(i,j)
            piece = self.get_piece_at_vector(position_vector)
            # skip if none
            if piece:
                yield piece, position_vector
            # if piece:
            #     dev_print(f"not skipping {position_vector}")
            #     yield piece, position_vector
            # else:
            #     dev_print(f"skipping {position_vector}")

    def generate_pieces_of_color(self, color=None):
        if color is None:
            color = self.next_to_go

        def same_color(data_item):
            piece, _ = data_item
            return piece.color == color

        yield from filter(
            same_color,
            self.generate_all_pieces()
        )

    def color_in_check(self, color=None):
        if color is None:
            color = self.next_to_go
            
        # it is now A's turn
        color_a = color
        color_b = "W" if color == "B" else "B"
        
        # we will examine all the movement vectors of B's pieces 
        # if any of them could take the A's King then currently A is in check as their king is threatened by 1 or more pieces (which could take it next turn)
        for piece, position_v in self.generate_pieces_of_color(color=color_b):
            movement_vs = piece.generate_movement_vectors(
                pieces_matrix=self.pieces_matrix,
                position_vector=position_v
            )
            for movement_v in movement_vs:
                resultant = position_v + movement_v
                to_square = self.get_piece_at_vector(resultant)
                # print(f"to_square   -->   {to_square!r}")
                # dev_print(f"piece at square {resultant.to_square()} is {to_square.symbol() if to_square else '<empty>'}")
                # dev_print(f"piece at square {resultant.to_square()} is {repr(to_square) if to_square else '<empty>'}")
                # As_move_threatens_king_A = isinstance(to_square, pieces_mod.King) and to_square.color == color_a
                As_move_threatens_king_A = (to_square == pieces_mod.King(color=color_a))
                # print(f"to_square == pieces_mod.King(color='B')   -->   {to_square!r} == {pieces_mod.King(color='B')!r}   -->    {to_square == pieces_mod.King(color='B')}")
                # dev_print(f"therefore king {'IS' if As_move_threatens_king_A else 'NOT'} threatened as square {'DOES' if As_move_threatens_king_A else 'DOES NOT'} contain {pieces_mod.King(color=color_a)!r} instead containing {repr(to_square) if to_square else '<empty>'}")
                # if As_move_threatens_king_A break out of all 3 loops
                if As_move_threatens_king_A:
                    # dev_print(f"color_in_check(color='{color}') returning True")
                    # dev_print(f"piece {piece.symbol()} at {position_v.to_square()} moving to {resultant.to_square()} IS threatening king")
                    return True
                # else:
                    # dev_print(f"piece {piece.symbol()} at {position_v.to_square()} moving to {resultant.to_square()} NOT threatening king")
        # dev_print(f"color_in_check(color='{color}') returning False")
        return False

    
    def generate_legal_moves(self):
        # dev_print(f"analysing legal moves for next to go {self.next_to_go}")
        for piece, piece_position_vector in self.generate_pieces_of_color(color=self.next_to_go):
            # dev_print(f"analysing moves for piece: {piece.symbol()}")
            movement_vectors = piece.generate_movement_vectors(
                pieces_matrix=self.pieces_matrix,
                position_vector=piece_position_vector
            )
            for movement_vector in movement_vectors:
                # dev_print(f"\tpiece {piece.symbol()}:_analysing movement vector {repr(movement_vector)}")
                child_game_state: Board_State = self.make_move(from_position_vector=piece_position_vector, movement_vector=movement_vector)
                # dev_print(f"\t\tmovement vector {repr(movement_vector)}: results in child game state")
                # child_game_state.print_board(
                #     print_function=lambda rows: list(map(
                #         lambda item: print(f"\t\t{item}"), 
                #         rows
                #     ))
                # )
                # dev_print(repr(child_game_state))
                is_check_after_move = child_game_state.color_in_check(color=self.next_to_go)
                dev_print(f"\t\tThis game state in check? for {self.next_to_go}:  {is_check_after_move}")

                if not is_check_after_move:
                    yield piece_position_vector, movement_vector

    # def generate_legal_moves(self):
    #     for piece, piece_position_vector in self.generate_pieces_of_color(color=self.next_to_go):
    #         movement_vectors = piece.generate_movement_vectors(
    #             pieces_matrix=self.pieces_matrix,
    #             position_vector=piece_position_vector
    #         )
    #         for movement_vector in movement_vectors:
    #             child_game_state = self.make_move(from_position_vector=piece_position_vector, movement_vector=movement_vector)
    #             is_check_after_move = child_game_state.color_in_check(color=self.next_to_go)
    #             if not is_check_after_move:
    #                 yield piece_position_vector, movement_vector

    def is_game_over_for_next_to_go(self):
        # check if in checkmate
        # for player a
        # if b has no moves
        if not list(self.generate_legal_moves()):
            # if b in check
            if self.color_in_check():
                # checkmate for b, a wins
                return True, self.next_to_go
            else:
                # stalemate
                return True, None

        return False, None

    def static_evaluation(self):
        """Give positive static evaluation if white is winning"""
        def generate_all_pieces():
            for i, j in iter_product(range(8), range(8)):
                piece_position_vector = Vector(i, j)
                piece: pieces_mod.Piece = self.get_piece_at_vector(piece_position_vector)
                if not piece:
                    continue

                yield piece, piece_position_vector
    
        over, winner = self.is_game_over_for_next_to_go()
        if over:
            match winner:
                case None: multiplier = 0
                case "W": multiplier = 1
                case "B": multiplier = -1
            return winner * ARBITRARILY_LARGE_VALUE

        else:
            return sum(piece.get_value(position_vector) * multiplier for piece, position_vector in generate_all_pieces())

    def make_move(self, from_position_vector: Vector, movement_vector: Vector):
        to_position_vector = from_position_vector + movement_vector
        # poor code, this below line can cause infinite recursion when legal moves generator called post check changes
        # assert (from_position_vector, movement_vector) in self.generate_legal_moves()

        new_pieces_matrix = deepcopy(self.pieces_matrix)
        # convert to list
        new_pieces_matrix = list(map(list, new_pieces_matrix))

        # set from to blank
        row, col = 7-from_position_vector.j, from_position_vector.i
        piece: pieces_mod.Piece = new_pieces_matrix[row][col]

        piece.last_move = movement_vector

        new_pieces_matrix[row][col] = None

        # set to square to this piece
        row, col = 7-to_position_vector.j, to_position_vector.i
        new_pieces_matrix[row][col] = piece

        # convert back to tuple
        new_pieces_matrix = tuple(map(tuple, new_pieces_matrix))
        new_next_to_go = "W" if self.next_to_go == "B" else "B"

        return Board_State(
            next_to_go=new_next_to_go,
            pieces_matrix=new_pieces_matrix
        )



In [3]:

# tested so assumed correct
import pieces
from vector import Vector


def test_dir(file_name): return f"test_data/board_state/{file_name}.yaml"

# code repeated from test pieces, opportunity to reduce redundancy


def list_map(function, iterable): return list(map(function, iterable))
def tuple_map(function, iterable): return tuple(map(function, iterable))


def descriptor_to_piece(descriptor) -> pieces.Piece:
    # converts WN to knight object with a color attribute of white
    if descriptor is None:
        return None

    color, symbol = descriptor
    piece_type: pieces.Piece = pieces.PIECE_TYPES[symbol]
    return piece_type(color=color)


def deserialize_pieces_matrix(pieces_matrix, next_to_go="W") -> Board_State:
    def row_of_symbols_to_pieces(row):
        return list_map(descriptor_to_piece, row)

    # update pieces_matrix replacing piece descriptors to piece objects
    pieces_matrix = list_map(row_of_symbols_to_pieces, pieces_matrix)
    board_state: Board_State = Board_State(pieces_matrix=pieces_matrix, next_to_go=next_to_go)

    return board_state


In [4]:

next_to_go = "B"
pieces_matrix = [
    [None, None, None, "BK", None, None, None, None],
    [None, None, None, "WP", None, None, None, None],
    [None, None, None, "WK", None, None, None, None],
    [None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None, None]
]
expected_legal_moves = []


In [5]:
board_state: Board_State = deserialize_pieces_matrix(pieces_matrix=pieces_matrix, next_to_go=next_to_go)


In [6]:
board_state.print_board()

[[None, None, None, 'BK', None, None, None, None],
 [None, None, None, 'WP', None, None, None, None],
 [None, None, None, 'WK', None, None, None, None],
 [None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None]]


In [7]:
set(board_state.generate_legal_moves())


		This game state in check? for B:  True
		This game state in check? for B:  True
		This game state in check? for B:  True
		This game state in check? for B:  True
		This game state in check? for B:  True


set()

In [8]:
board_state = Board_State(
    next_to_go="W",
    pieces_matrix= [
        [None, None, pieces.King(color="B"), None, None, None, None, None],
        [None, None, None, pieces.Pawn(color="W"), None, None, None, None],
        [None, None, None, pieces.King(color="W"), None, None, None, None],
        [None, None, None, None, None, None, None, None],
        [None, None, None, None, None, None, None, None],
        [None, None, None, None, None, None, None, None],
        [None, None, None, None, None, None, None, None],
        [None, None, None, None, None, None, None, None]
    ]
)

In [9]:
board_state.color_in_check("W")

False

In [10]:
board_state.color_in_check("B")

True

In [11]:
pieces.King("B") == pieces.King("B")

True